In [0]:

spark.read.parquet("s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.download-attribution.v3/fact/granularity=daily/date=2020-04-01/device_code=ios-phone/").printSchema()


In [0]:
%%sh
# aws s3 ls s3://b2c-prod-data-pipeline-unified-review/review.v4/fact/

# aws s3 ls s3://b2c-prod-data-pipeline-unified-review/unified/review.v4/fact/process_granularity=daily/process_date=2020-09-08/process_hour=23/device_code=ios-all/market_code=apple-store/ | sort -n | head -10





# aws s3 ls s3://b2c-prod-data-pipeline-unified-review/unified/review.v4/fact/

date

# aws s3 ls s3://prod_appannie_uniform_public_app_store_data/review-uniform/DAILY/2020-09-08/23/gp/  | head -10
aws s3 ls s3://prod_appannie_uniform_public_app_store_data/review-uniform/HOURLY/2020-09-09/ --recursive | grep 284882215

# aws s3 ls s3://prod_appannie_uniform_public_app_store_data/review-uniform/DAILY/2020-09-08/23/ios/544007664.gz 



 

In [0]:

# spark.read.csv("s3://prod_appannie_uniform_public_app_store_data/review-uniform/DAILY/2020-09-07/23/ios/284882215.gz", sep = "\t").orderBy("_c11", ascending=False).show(20)
spark.read.csv("s3://prod_appannie_uniform_public_app_store_data/review-uniform/HOURLY/2020-08-01/16/ios/284882215.gz", sep = "\t").orderBy("_c11", ascending=False).show(20)


In [0]:




spark.read.format("delta").load("s3://b2c-prod-data-pipeline-unified-review/unified/review.v4/fact").where("process_granularity='daily' and process_date='2020-09-10' and process_hour=23 and device_code='ios-all' and market_code='apple-store'").orderBy("time", ascending=False).show(300)


In [0]:

df = spark.read.option('basePath','s3://b2c-prod-data-pipeline-qa/aa.review/').parquet("s3://b2c-prod-data-pipeline-qa/aa.review/{event_time_repartition_20200116_20200224_count,event_time_repartition_streaming_20200220_20200310_count,event_time_union_data_200806_20200116_count}")
result =  df.groupBy("event_month").agg({"agg_count":"sum"}).cache().collect()



In [0]:

spark.read.format("delta").load("s3://b2c-prod-data-pipeline-unified-advancedreview/unified/advancedreview.topic.v1/fact/").where("process_date = '2020-07-28'").createOrReplaceTempView("adv_data")
spark.read.format("delta").load("s3://b2c-prod-data-pipeline-unified-review/unified/review.v4/fact/").where("process_date = '2020-07-28'").createOrReplaceTempView("review_data")
spark.sql("""
select * from adv_data a left join review_data rv
on a.review_id = rv.review_id 
where rv.review_id is null
""").show()

In [0]:

from pyspark.sql.utils import AnalysisException

from aadatapipelinecore.core.fs.device import s3
from aadatapipelinecore.core.fs import Conf
import json
import time
from requests.auth import HTTPBasicAuth
from aadatapipelinecore.core.utils.http import plain_get


def es_doc(month):
    month = month.replace("-","")
    headers = {
        'content-type': 'application/json'
    }
    auth = HTTPBasicAuth("bdp", "C38vEJEuraCw")
    ios_url = "http://internal-internel-review-es-v7-1286999769.us-east-1.elb.amazonaws.com:19200/int-ss-review_v1-*-{}/_count".format(
        month)
    ios_resp = plain_get(ios_url, timeout=30, auth=auth, headers=headers, is_resource_of_appannie=False)
    ios_doc = json.loads(ios_resp.text)
    return ios_doc["count"]

for x in result:
    db_count = es_doc(x[0])
    if db_count != x[1]:
        print "not equal , month is {},  unified is {},  db is {}, diff is {} ".format(x[0], x[1],db_count,db_count- x[1])



In [0]:

from aadatapipelinecore.core.log import logger
import unittest
from pyspark.sql.utils import AnalysisException
from aadatapipelinecore.core.fs.device import s3
from aadatapipelinecore.core.fs import Conf
import datetime
from pyspark.sql.functions import year, month, dayofmonth
from pyspark.sql.functions import date_format


df = spark.read.option('basePath','s3://b2c-prod-data-pipeline-unified-review/unified/review.v1/fact/').parquet("s3://b2c-prod-data-pipeline-unified-review/unified/review.v1/fact/process_granularity={hourly,daily,weekly}/process_date={2020-01-16,2020-01-17,2020-01-18,2020-01-19,2020-01-20,2020-01-21,2020-01-22,2020-01-23,2020-01-24,2020-01-25,2020-01-26,2020-01-27,2020-01-28,2020-01-29,2020-01-30,2020-01-31,2020-02-01,2020-02-02,2020-02-03,2020-02-04,2020-02-05,2020-02-06,2020-02-07,2020-02-08,2020-02-09,2020-02-10,2020-02-11,2020-02-12,2020-02-13,2020-02-14,2020-02-15,2020-02-16,2020-02-17,2020-02-18,2020-02-19,2020-02-20,2020-02-21,2020-02-22,2020-02-23,2020-02-24}").where("market_code in ('apple-store','google-play') and device_code in ('android-all', 'ios-all')").cache()
# df =spark.read.format("delta").load("s3://b2c-prod-data-pipeline-unified-review/unified/review.v4/fact/").where("(process_date between '2020-02-20' and '2020-03-09') and market_code in ('apple-store','google-play') and device_code in ('android-all', 'ios-all')").cache()
df = df.withColumn("time",df["time"].cast('date'))
new_df = df.withColumn("event_month",date_format('time', 'yyyy-MM'))
result =  new_df.groupBy("event_month").agg({"*":"count"}).cache().collect()
for x in result:
    c_temp_load=spark.read.parquet("s3://b2c-prod-data-pipeline-unified-review/_obsolete/review_load_temp/review_load_temp_streaming_20200220_20200310/event_month={}".format(x[0])).filter("process_date != '2020-03-10' and market_code in ('apple-store','google-play') and device_code in ('android-all', 'ios-all') ").count()
    if x[1] != c_temp_load:
    # if x[1] != spark.read.parquet("s3://b2c-prod-data-pipeline-unified-review/_obsolete/review_load_temp/review_load_temp_20200116_20200224/event_month={}".format(x[0])).count():

        print 'falied  month is {},  v4 unified is : {}, temp load data is {}, diff is {}  '.format( x[0], x[1], c_temp_load, c_temp_load-x[1] )
    else:
        print 'pass', x[0]

In [0]:

from pyspark.sql.utils import AnalysisException

from aadatapipelinecore.core.fs.device import s3
from aadatapipelinecore.core.fs import Conf
import json
import time
from requests.auth import HTTPBasicAuth
from aadatapipelinecore.core.utils.http import plain_get

print result[0][0]
for x in result:
    c1 = spark.read.parquet("s3://b2c-prod-data-pipeline-unified-review/_obsolete/review_load_temp/review_load_temp_streaming_20200220_20200310/event_month={}".format(x[0])).select("review_id").distinct().count()
    c2 = spark.read.parquet("s3://b2c-prod-data-pipeline-unified-review/_obsolete/review_load_aggr/review_load_aggr_streaming_20200220_20200310/event_month={}".format(x[0])).select("review_id").distinct().count()
    # c3 = spark.read.parquet("s3://b2c-prod-data-pipeline-unified-review/_obsolete/review_load_aggr/review_load_aggr_20200116_20200224/event_month={}".format(x[0])).select("review_id").distinct().count()
    # spark.createDataFrame([(x[0], c3)] , schema=["event_month","agg_count"]).coalesce(1).write.mode("append").parquet("s3://b2c-prod-data-pipeline-qa/aa.review/event_time_repartition_20200116_20200224_count/")
    # spark.createDataFrame([(month, unified_count)], schema=["event_month","agg_count"]).coalesce(1).write.mode("append").parquet("s3://b2c-prod-data-pipeline-qa/aa.review/event_time_union_data_200806_20200116_count/")
    if c1 != c2:
        spark.createDataFrame(result).withColumnRenamed("count(1)","agg_count").coalesce(1).write.mode("append").parquet("s3://b2c-prod-data-pipeline-qa/aa.review/event_time_repartition_streaming_20200220_20200310_count/")

        print 'failed', x[0]
    else:
        print 'pass',x[0]


In [0]:

from aadatapipelinecore.core.log import logger
import unittest
from pyspark.sql.utils import AnalysisException
from aadatapipelinecore.core.fs.device import s3
from aadatapipelinecore.core.fs import Conf
source_path='s3://b2c-prod-data-pipeline-unified-review/_obsolete/review_load_aggr/review_load_aggr_streaming_20200220_20200310/'
dest_path='s3://b2c-prod-data-pipeline-unified-review/_obsolete/bulk_file/bulk_file_streaming_20200220_20200310/date={}-01/'
s3_bucket_list = s3.S3Bucket(Conf(bucket_name='b2c-prod-data-pipeline-unified-review'))
path_list = s3_bucket_list.all(prefix="_obsolete/review_load_aggr/review_load_aggr_streaming_20200220_20200310/", depth_is_1=True)
print path_list[0]
# print path_list[129]
for path in path_list:
    # print path.split("/")[3].split("=")[1]
    count_1 = spark.read.parquet("s3://b2c-prod-data-pipeline-unified-review/{}".format(path)).select("review_id").distinct().count()
    df = spark.read.json(dest_path.format(path.split("/")[3].split("=")[1])).cache()
    count_2 = df.filter("index is not null").count()
    count_3 = df.filter("_identifier is not null").count()
    # print count_1
    if count_1 == count_2 == count_3 :
        print 'pass', path, count_1
    else:
        print "not equal!!!!!!", count_1 , count_2, count_3,  path



In [0]:


for x in dup_list:
    result1 = spark.read.parquet('s3://b2c-prod-data-pipeline-unified-review/unified/review_load_temp/review_load_temp_v4/event_month=2019-09').filter("review_id='{}'".format(x["review_id"])).groupBy("process_date","process_hour","time").agg({"*":"count"}).orderBy(desc("time"),desc("process_date"),desc("process_hour")).take(1)
    result2 = spark.read.parquet("s3://b2c-prod-data-pipeline-unified-review/unified/review_load_temp/review_agg_v1/event_month=2019-09").filter("review_id='{}'".format(x["review_id"])).take(1)
    print result1,result2
    if result1[0]["process_date"]==result2[0]["process_date"] and result1[0]["process_hour"]==result2[0]["process_hour"] and result1[0]["time"]==result2[0]["time"]  :
        print 'pass'
    else:
        print 'failed!!!',result1, result2,x

for x in process_data_set:
    path = "s3://b2c-prod-data-pipeline-unified-review/unified/review.v1/fact/process_granularity={weekly,daily,hourly}/process_date=%s/" % x["process_date"].strftime("%Y-%m-%d")
    total_count = spark.read.option("basePath","s3://b2c-prod-data-pipeline-unified-review/unified/review.v1/fact/").parquet(path).where("market_code in ('apple-store','google-play')").count()
    if x["total_review"] != total_count:
       print "Not Equal!!!!!!!!", x["process_date"].strftime("%Y-%m-%d") ,x["total_review"], total_count
    else:
        print x["process_date"].strftime("%Y-%m-%d") ,x["total_review"],total_count

In [0]:

from pyspark.sql.utils import AnalysisException

from aadatapipelinecore.core.fs.device import s3
from aadatapipelinecore.core.fs import Conf
import json
import time
from requests.auth import HTTPBasicAuth
from aadatapipelinecore.core.utils.http import plain_get


s3_bucket_list = s3.S3Bucket(Conf(bucket_name='b2c-prod-data-pipeline-unified-review'))
path_list = s3_bucket_list.all(prefix="unified/review_load_temp/review_agg_v3/", depth_is_1=True)
print len(path_list)

def get_month_count(path):
    df1 = spark.read.parquet("s3://b2c-prod-data-pipeline-unified-review/{}".format(path)).select("review_id")
    

    month=path.split("=")[1].replace("/","")
    print month
    review_path_list=[""]
    try:
        multiple_path = "s3://b2c-prod-data-pipeline-unified-review/_obsolete/review_load_aggr/[review_load_aggr_20200116_20200224,review_load_aggr_streaming_20200220_20200310,review_load_aggr_streaming_20200310_20200312]/event_month=%s" % month
        print multiple_path
        df2 = spark.read.option('basePath','s3://b2c-prod-data-pipeline-unified-review/_obsolete/review_load_aggr/').parquet(multiple_path)
        df3 = spark.read.parquet("s3://b2c-prod-data-pipeline-unified-review/unified/review.v1/fact/process_granularity=monthly/process_date={}*/".format(month)).filter("market_code in ('apple-store','google-play')").select("review_id")
        final_df = df1.union(df2).union(df3)
        return month, final_df.select("review_id").distinct().count()
    except AnalysisException:
        print 'no monthly data'.format(month)
        return month, df1.select("review_id").distinct().count()
        

# df.coalesce(1).write.mode("append").parquet("s3://b2c-prod-data-pipeline-qa/aa.review/historical_data")


def es_doc(month):
    month = month.replace("-","")
    headers = {
        'content-type': 'application/json'
    }
    auth = HTTPBasicAuth("bdp", "C38vEJEuraCw")
    ios_url = "http://internal-internel-review-es-v7-1286999769.us-east-1.elb.amazonaws.com:19200/int-ss-review_v1-*-{}/_count".format(
        month)
    ios_resp = plain_get(ios_url, timeout=30, auth=auth, headers=headers, is_resource_of_appannie=False)
    ios_doc = json.loads(ios_resp.text)
    return ios_doc["count"]


for path in path_list[::-1]:
    month, unified_count = get_month_count(path)
    print month, unified_count
    # spark.createDataFrame([(month, unified_count)], schema=["event_month","agg_count"]).coalesce(1).write.mode("append").parquet("s3://b2c-prod-data-pipeline-qa/aa.review/event_time_union_data_200806_20200116_count/")
    # # spark.createDataFrame([month, unified_count]).coalesce(1).write.mode("append").parquet("s3://b2c-prod-data-pipeline-qa/aa.review/event_time_repartition_streaming_20200220_20200310_count/")
    db_count = es_doc(month)
    if unified_count!= db_count:
        print 'not equal!!!! month {}, unified count is {}, db count is {}, db-unified diff is {} '.format(month, unified_count, db_count, db_count-unified_count)
    else:
        print 'month {}, unified count is {}, db count is {}'.format(month, unified_count, db_count)

In [0]:

spark.read.parquet("s3://b2c-prod-data-pipeline-qa/aa.review/event_time_repartition_20200116_20200224_count/").printSchema()

In [0]:
%%sh
# aws s3 ls s3://b2c-prod-data-pipeline-unified-review/_obsolete/bulk_file/bulk_file_streaming_20200220_20200310/
# aws s3 ls s3://b2c-prod-data-pipeline-unified-review/unified/review.v4/fact/process_granularity=daily/process_date=2020-01-31/process_hour=23/device_code=ios-all/market_code=apple-store/
# aws s3 ls s3://b2c-prod-data-pipeline-qa/aa.review/test_03_12/
# aws s3 ls s3://b2c-prod-data-pipeline-unified-review/unified/review_load_temp/review_agg_v3/event_month=2011-12/
aws s3 ls s3://b2c-prod-data-pipeline-qa/aa.review/



In [0]:

# spark.read.json("s3://b2c-prod-data-pipeline-unified-review/unified/review.v1/fact/").show()
review_df = spark.read.format("delta").load("s3://b2c-prod-data-pipeline-unified-review/unified/review.v4/fact").where("process_granularity in ('daily','weekly','hourly') and process_date between '2020-02-20' and '2020-03-06' and process_hour between 0 and 23 and device_code='ios-all' and market_code='apple-store'")
# unified_df1= unified_df1.withColumn("event_date",unified_df1["time"].cast('date'))

review_df = review_df.withColumn("time",review_df["time"].cast('date')).repartition("time")




In [0]:

from pyspark.sql.types import StringType

spark.createDataFrame([('2020-01', "10")], schema=["event_month","agg_count"]).show()

In [0]:

from aadatapipelinecore.core.log import logger

import unittest
from pyspark.sql.utils import AnalysisException
from aadatapipelinecore.core.fs.device import s3
from aadatapipelinecore.core.fs import Conf

s3_bucket_list = s3.S3Bucket(Conf(bucket_name='b2c-prod-data-pipeline-unified-review'))
path_list = s3_bucket_list.all(prefix="_obsolete/bulk_file/bulk_file_streaming_20200220_20200310/", depth_is_1=True)

result_set=set()

print path_list[0]
print len(path_list)
for path in path_list:
    
    index_list = spark.read.json("s3://b2c-prod-data-pipeline-unified-review/{}".format(path)).filter("index is not null").select("index").collect()
    # print len(index_list)
    for x in range(0, len(index_list)):
        result_set.add(index_list[x]["index"]["_id"])
        
print len(result_set)
# index_list = spark.read.json("s3://b2c-prod-data-pipeline-unified-review/{}".format()).filter("index is not null").select("index")
# index_list.show()
# data_set = index_list.collect()
# result_set=set()
# review_id_list_month =[ x["index"] for x in result_month]


In [0]:

from aadatapipelinecore.core.log import logger

import unittest
from pyspark.sql.utils import AnalysisException
from aadatapipelinecore.core.fs.device import s3
from aadatapipelinecore.core.fs import Conf

source_path = 's3://b2c-prod-data-pipeline-unified-review/unified/review.v1/fact/process_granularity=monthly/'
dest_path = 's3://b2c-prod-data-pipeline-unified-review/_obsolete/review_load_aggr/review_load_aggr_20200116_20200224/date={}'

s3_bucket_list = s3.S3Bucket(Conf(bucket_name='b2c-prod-data-pipeline-unified-review'))
path_list = s3_bucket_list.all(prefix="unified/review.v1/fact/process_granularity=monthly/event_month=", depth_is_1=True)
print path_list[0]
for path in path_list[155:]:
    print path.split("/")[4].split("=")[1]

    count_1 = spark.read.parquet("s3://b2c-prod-data-pipeline-unified-review/{}".format(path)).where(
        "market_code in ('apple-store','google-play')").select("review_id").distinct().count()
    new_path = path.split("/")[4].split("=")[1]
    df = spark.read.json(dest_path.format(new_path)).cache()
    count_2 = df.filter("index is not null").count()
    count_3 = df.filter("_identifier is not null").count()

    # print count_1
    if count_1 == count_2 == count_3:
        pass
        # print 'pass', path
    else:
        print "not equal!!!!!!", count_1, count_2, count_3, path


In [0]:

from aadatapipelinecore.core.fs.device import s3
from aadatapipelinecore.core.fs import Conf
import json
import time
from requests.auth import HTTPBasicAuth
from aadatapipelinecore.core.utils.http import plain_get

# source_path = 's3://b2c-prod-data-pipeline-unified-review/unified/review_bulk_json.v3/'
#
# source_path = 's3://b2c-prod-data-pipeline-unified-review/unified/review.v1/fact/process_granularity=monthly/'
# dest_path = 's3://b2c-prod-data-pipeline-unified-review/unified/review_bulk_json_month.v3/date={}'


s3_bucket_list = s3.S3Bucket(Conf(bucket_name='b2c-prod-data-pipeline-unified-review'))
path_list = s3_bucket_list.all(prefix="unified/review_bulk_json_month.v3/", depth_is_1=True)
path_list = path_list[8:40]
print path_list


def get_total_distinct_month_data( path):
    df = spark.read.json("s3://b2c-prod-data-pipeline-unified-review/{}".format(path)).filter("index is not null")
    result_month = df.collect()
    review_id_list_month =[ x["index"] for x in result_month]

    bulk_date = path.split("=")[-1][:7]
    df2 = spark.read.json("s3://b2c-prod-data-pipeline-unified-review/unified/review_bulk_json.v3/date={}-01/".format(bulk_date)).filter("index is not null")
    review_id_bulk = df2.collect()
    review_id_list_bulk=[ x["index"] for x in review_id_bulk]

    review_month = spark.createDataFrame(review_id_list_month)
    review_bulk = spark.createDataFrame(review_id_list_bulk)

    new_df = review_month.join(review_bulk, "_id", "outer")
    count = new_df.select("_id").distinct().count()
    return count

def get_path():
    index_list = []
    for path in path_list:
        # total = spark.read.json("s3://b2c-prod-data-pipeline-unified-review/{}".format(path)).filter(
        #     "_identifier is not null").count()
        total = get_total_distinct_month_data(path)

        new_path = path.split("=")[1][:7].replace("-", "")

        index_list.append(("int-ss-review_v1-apple-store-ios-all-{}".format(new_path),
                           "int-ss-review_v1-google-play-android-all-{}".format(new_path), total))
    return index_list


def get_es_result(index):
    # print index
    headers = {
        'content-type': 'application/json'
    }
    auth = HTTPBasicAuth("bdp", "C38vEJEuraCw")
    url = "http://internal-internel-review-es-v7-1286999769.us-east-1.elb.amazonaws.com:19200/{}/_count".format(
        index)
    try:
        resp = plain_get(url, timeout=30, auth=auth, headers=headers, is_resource_of_appannie=False)
        doc = json.loads(resp.text)
        return doc["count"]
    except KeyError:
        print 'this index is not exist ', index
        return 0


def es_doc(index):
    print 'es doc'
    total_count = get_es_result(index[0]) + get_es_result(index[1])
    print total_count, index[2]
    if total_count != index[2]:
        print 'not equal! path : {} unified count: {} , db count: {}'.format(index[0], index[2], total_count)
    else:
        print index[0], index[1], total_count


def compare(index_list):
    for row in index_list:
        es_doc( row)
        # compare(doc, row)


path_list_1 = get_path()
print path_list_1
compare( path_list_1)



In [0]:


# granularity="hourly"
# date='2020-03-06'
# path_ = '''
#         process_granularity='{}' and process_date='{}' and market_code in ('apple-store','google-play')
#         and device_code in ('android-all', 'ios-all')'''.format(granularity, date)

# unified_df1 = spark.read.format("delta").load(
#             "s3://b2c-prod-data-pipeline-unified-review/unified/review.v4/fact/").cache().where("process_granularity='daily' and process_date='2020-03-06' and process_hour='23' and device_code='android-all' and market_code='google-play' ")
unified_df1=spark.read.parquet("s3://b2c-prod-data-pipeline-unified-review/unified/review.v4/fact/process_granularity=daily/process_date=2020-03-06/process_hour=23/device_code=android-all/market_code=google-play/part-00000-c374f6b1-ca8b-4f71-9ba9-2b4431c52ce1.c000.gz.parquet")
# unified_df1= unified_df1.withColumn("event_date",unified_df1["time"].cast('date'))
unified_df1.createOrReplaceTempView("temp_unified")


In [0]:

# unified_df1.show()
spark.sql("select DISTINCT review_id from (select *, cast(time as date) as converted_date from temp_unified ) as converted  where converted_date='{}' ".format('2020-03-06')).show()

In [0]:
%%sh
# curl -XGET -u bdp:C38vEJEuraCw  -H 'Content-Type: application/json'   http://internal-internel-review-es-v7-1286999769.us-east-1.elb.amazonaws.com:19200/int-ss-review_v1-apple-store-ios-all-202002/_mapping?pretty
curl -XGET -u bdp:C38vEJEuraCw  -H 'Content-Type: application/json'   http://internal-internel-review-es-v7-1286999769.us-east-1.elb.amazonaws.com:19200/int-ss-review_v1-apple-store-ios-all-2019*/_alias?pretty
curl -X GET -u bdp:"C38vEJEuraCw" "http://internal-internel-review-es-v7-1286999769.us-east-1.elb.amazonaws.com:19200/int-ss-review_v1*202009*/_mapping/field/content?pretty"



In [0]:
%%sh
curl -XGET -u  bdp_read:E9vjhU2qG1bAcM83  -H 'Content-Type: application/json'  http://internal-aa-int-ss-review-elasticsearch-126620990.us-east-1.elb.amazonaws.com:19200/int-reviews-ios-202002/ios/5549941283?routing=1439363292

curl -XGET -u bdp_read:E9vjhU2qG1bAcM83  -H 'Content-Type: application/json'   http://internal-aa-int-ss-review-elasticsearch-126620990.us-east-1.elb.amazonaws.com:19200/int-reviews-ios-202009*/ios/_search?pretty -d '{
  "query": {
    "match": {
    "date":"2020-09-08T00:00:00.000000Z"
    }
  }
}'



In [0]:

# review_id = 4945989288

# {"query":{"match":{"_id":"4945989288"}}}
import json
import time
from requests.auth import HTTPBasicAuth
from aadatapipelinecore.core.utils.http import plain_get

def es_doc():
    headers = {
        'content-type': 'application/json'
    }
    auth = HTTPBasicAuth("bdp_read","E9vjhU2qG1bAcM83")
    
    url = '''http://internal-aa-int-ss-review-elasticsearch-126620990.us-east-1.elb.amazonaws.com:19200/int-reviews-ios-201910/_search?q=_id:5474203681'''
    resp = plain_get(url, timeout=30, auth=auth, headers=headers, is_resource_of_appannie=False)
    doc = json.loads(resp.text)
    print doc
es_doc()


In [0]:

from elasticsearch import Elasticsearch

def es_doc(identifier):
    # print 'review_id ', review_id
    es_connection = Elasticsearch(["http://internal-internel-review-es-v7-1286999769.us-east-1.elb.amazonaws.com:19200"],http_auth=("bdp_qa","6trmJcZBJrKWdtN2"),port=19200)
    doc = es_connection.count(index="int-ss-review_v1-*", body={"query":{"match":{"_identifier":"{}".format(identifier)}}})
    return doc["count"]
print es_doc("220200211082455980")

In [0]:
%%sh

curl -XGET -u bdp:C38vEJEuraCw  -H 'Content-Type: application/json'   http://internal-internel-review-es-v7-1286999769.us-east-1.elb.amazonaws.com:19200/int-ss-review_v1-google-play-android-all*/_search?pretty -d '{
  "query": {
    "match": {
      "_id": "gp:AOqpTOFuW2_xRLOefHTGXc8Q8XuPS9GoYj3_Ybq6Zh3ci3EnmN9ug8ixmPMEnH8jh1KSlekTAu4cnF6R6KNLTg"
    }
  }
}'

# curl -XGET -u bdp_read:E9vjhU2qG1bAcM83  -H 'Content-Type: application/json'   http://internal-aa-int-ss-review-elasticsearch-126620990.us-east-1.elb.amazonaws.com:19200/int-reviews-ios-202002/_search?pretty -d '{
# "match":{"_id":"gp:AOqpTOFuW2_xRLOefHTGXc8Q8XuPS9GoYj3_Ybq6Zh3ci3EnmN9ug8ixmPMEnH8jh1KSlekTAu4cnF6R6KNLTg"}
# }'



In [0]:
%%sh
curl -XGET -u bdp_read:E9vjhU2qG1bAcM83  -H 'Content-Type: application/json'   http://internal-aa-int-ss-review-elasticsearch-126620990.us-east-1.elb.amazonaws.com:19200/int-reviews-ios-202002/_search?pretty -d '{
"query":{"range":{"date":{"gte":"2020-02-26","lte":"2020-02-26"}}}}'

echo 'hello hello'

curl -XGET -u bdp:C38vEJEuraCw  -H 'Content-Type: application/json'   http://internal-internel-review-es-v7-1286999769.us-east-1.elb.amazonaws.com:19200/int-ss-review_v1-google-play-android-all-202002/_count?pretty -d '{
"query":{"range":{"date":{"gte":"2020-02-26","lte":"2020-02-26"}}}}'



In [0]:

# review_id = 4945989288

# {"query":{"match":{"_id":"4945989288"}}}
import json
import time
from requests.auth import HTTPBasicAuth
from aadatapipelinecore.core.utils.http import plain_get

def es_doc():
    headers = {
        'content-type': 'application/json'
    }
    auth = HTTPBasicAuth("bdp_read","E9vjhU2qG1bAcM83")
    
    url = '''http://internal-aa-int-ss-review-elasticsearch-126620990.us-east-1.elb.amazonaws.com:19200/int-reviews-ios-201910/_search?q=_id:5474203681'''
    resp = plain_get(url, timeout=30, auth=auth, headers=headers, is_resource_of_appannie=False)
    doc = json.loads(resp.text)
    print doc
es_doc()


In [0]:
%python
import json
import time
from requests.auth import HTTPBasicAuth
from aadatapipelinecore.core.utils.http import plain_get


index = [("int-ss-review_v1-google-play-android-all","bdp","C38vEJEuraCw" ,"http://internal-internel-review-es-v7-1286999769.us-east-1.elb.amazonaws.com:19200"), ( "int-reviews-gp" , "bdp_read","E9vjhU2qG1bAcM83","http://internal-aa-int-ss-review-elasticsearch-126620990.us-east-1.elb.amazonaws.com:19200" ) ,("int-ss-review_v1-apple-store-ios-all" , "bdp","C38vEJEuraCw" ,"http://internal-internel-review-es-v7-1286999769.us-east-1.elb.amazonaws.com:19200"), ( "int-reviews-ios" , "bdp_read","E9vjhU2qG1bAcM83","http://internal-aa-int-ss-review-elasticsearch-126620990.us-east-1.elb.amazonaws.com:19200" )]

result=[]
def es_doc(index):
    headers = {
        'content-type': 'application/json'
    }
    auth = HTTPBasicAuth(x[1],x[2])
    
    url = "{}/{}-202003*/_count".format(index[3], index[0])
    resp = plain_get(url, timeout=30, auth=auth, headers=headers, is_resource_of_appannie=False)
    doc = json.loads(resp.text)
    return doc["count"]

for x in index:
    print x[0] , es_doc(x)
    result.append((x[0], es_doc(x)))

print int(result[0][1]) - int(result[1][1])
print int(result[2][1]) - int(result[3][1])

In [0]:

from aadatapipelinecore.core.fs.device import s3
from aadatapipelinecore.core.fs import Conf
import json
import time
from requests.auth import HTTPBasicAuth
from aadatapipelinecore.core.utils.http import plain_get

source_path = 's3://b2c-prod-data-pipeline-unified-review/unified/review_bulk_json_month.v3/'

s3_bucket_list = s3.S3Bucket(Conf(bucket_name='b2c-prod-data-pipeline-unified-review'))
path_list = s3_bucket_list.all(prefix="unified/review_bulk_json_month.v3/", depth_is_1=True)
print path_list[2].split("=")[1].replace("-","").strip("/")[:-2]

def es_doc(month):
    headers = {
        'content-type': 'application/json'
    }
    auth = HTTPBasicAuth("bdp","C38vEJEuraCw")
    url = "http://internal-internel-review-es-v7-1286999769.us-east-1.elb.amazonaws.com:19200/int-ss-review_v1-*-all-{}*/_count".format(month.split("=")[1].replace("-","").strip("/")[:-2])
    resp = plain_get(url, timeout=30, auth=auth, headers=headers, is_resource_of_appannie=False)
    doc = json.loads(resp.text)
    print doc
    
for x in path_list:
    es_doc(x)


In [0]:

df = spark.read.csv("s3://prod_appannie_uniform_public_app_store_data/review-uniform/DAILY/2019-10-15/23/ios/1147074917.gz",sep="\t").filter("_c0='4945989288'").show(20,False)
# spark.read.parquet("s3://b2c-prod-data-pipeline-unified-review/unified/review.v1/fact/process_granularity=daily/process_date=2019-12-01/").filter("app_id='1147074917' and review_id=5218497980").show(20,False)


# path = "s3://b2c-prod-data-pipeline-unified-review/unified/review.v1/fact/process_granularity={weekly,daily,hourly}/process_date=%s/" % x["process_date"].strftime("%Y-%m-%d")
# total_count = spark.read.option("basePath","s3://b2c-prod-data-pipeline-unified-review/unified/review.v1/fact/").parquet(path).where("market_code in ('apple-store','google-play')").count()

# print spark.read.csv("s3://prod_appannie_uniform_public_app_store_data/review-uniform/DAILY/2019-12-01").count()
# print spark.read.parquet("s3://b2c-prod-data-pipeline-unified-review/unified/review.v1/fact/process_granularity=daily/process_date=2019-12-01/").count()


In [0]:

from aadatapipelinecore.core.fs.device import s3
from aadatapipelinecore.core.fs import Conf
import json
import time
from requests.auth import HTTPBasicAuth
from aadatapipelinecore.core.utils.http import plain_get

s3_bucket_list = s3.S3Bucket(Conf(bucket_name='b2c-prod-data-pipeline-unified-review'))
path_list = s3_bucket_list.all(prefix="unified/review_bulk_json_month.v3/", depth_is_1=True)
path_list = path_list[:102]
print path_list[0]

from pyspark.sql.functions import *
df = spark.read.json("s3://b2c-prod-data-pipeline-unified-review/unified/review_bulk_json_month.v3/date=2008-07-31/").filter("index is not null") 
result_month = df.collect()
review_id_list_month =[ x["index"] for x in result_month]


df2 = spark.read.json("s3://b2c-prod-data-pipeline-unified-review/unified/review_bulk_json.v3/date=2008-07-01/").filter("index is not null")
review_id_bulk = df2.collect()
review_id_list_bulk=[ x["index"] for x in review_id_bulk]

review_month = spark.createDataFrame(review_id_list_month)
review_bulk = spark.createDataFrame(review_id_list_bulk)

new_df = review_month.join(review_bulk, "_id", "outer")

print new_df.select("_id").distinct().count()

In [0]:

columns = {'review_id', 'process_granularity', 'process_date', 'process_hour', 'market_code', 'device_code', 'app_id', 'product_version', 'user_id', 'user_name', 'user_language', 'user_device', 'user_purchased', 'user_review_url', 'rating', 'country_code', 'date', 'language', 'title', 'title_language', 'content', 'content_language', 'reply_id', 'reply', 'reply_date', 'reply_language', 'chinese_content', 'chinese_reply', 'chinese_title', 'english_content', 'english_reply', 'english_title', 'japanese_content', 'japanese_reply', 'japanese_title', 'korean_content', 'korean_reply', 'korean_title', 'other_language_content', 'other_language_reply', 'other_language_title', '_identifier'}
ignored_columns = {'process_granularity', 'process_date', 'process_hour','user_review_url','reply_id'}
columns -= ignored_columns
from pyspark.sql.functions import desc
from pyspark.sql import functions as F, types as T
unified_df = spark.read.option('basePath','s3://b2c-prod-data-pipeline-unified-review/unified/review.v1/fact/').parquet("s3://b2c-prod-data-pipeline-unified-review/unified/review.v1/fact/process_granularity={daily,hourly,weekly}/process_date=2020-01-25/").where("market_code in ('apple-store','google-play') and device_code in ('android-all', 'ios-all')")
dup_list = unified_df.select('review_id','process_date','process_hour').distinct().groupBy("review_id").agg({"*":"count"}).where("count(1)>1").orderBy("count(1)",ascending = False).take(10)
df = spark.read.json("s3://b2c-prod-data-pipeline-unified-review/unified/review_bulk_json_by_process_date.v3/date=2020-01-25/").withColumn("id_1",(F.monotonically_increasing_id()/2).cast("long"))
bulk_df = df.groupBy("id_1").agg(*(F.max(column).alias(column) for column in df.columns if column != "id_1"))
print bulk_df.show()
try:
    bulk_df = bulk_df.withColumn("review_id",bulk_df["index"]["_id"]).drop("id_1").drop("index").persist()
    for x in dup_list:
        try:
            print x["review_id"]
            result1_df = unified_df.filter("review_id='{}'".format(x["review_id"])).orderBy(desc("process_date"), desc("time"), desc("process_hour")).withColumn("date", F.from_unixtime(F.unix_timestamp("time", "yyyy-MM-dd"), "yyyy-MM-dd")).persist()
            result1 = result1_df.limit(1).take(1)
            result2 = bulk_df.filter("review_id='{}'".format(x["review_id"])).take(1)
            validate_df = result1_df.where((result1_df["process_date"] == result1[0]["process_date"]) & (result1_df["time"] == result1[0]["time"]) & (result1_df["process_hour"] == result1[0]["process_hour"]))
            for item in columns:
                words = item.split("_")
                if "chinese_" in item:
                    if result2[0][item] is not None:
                        validate_df = validate_df.where("%s_language in ('zh','zh_Hant')" % words[-1])
                elif "korean_" in item:
                    if result2[0][item] is not None:
                        validate_df = validate_df.where("%s_language in ('ko')" % words[-1])
                elif "english_" in item:
                    if result2[0][item] is not None:
                        validate_df = validate_df.where("%s_language in ('en')" % words[-1])
                elif "japanese_" in item:
                    if result2[0][item] is not None:
                        validate_df = validate_df.where("%s_language in ('ja')" % words[-1])
                elif "other_language_" in item:
                    if result2[0][item] is not None:
                        validate_df = validate_df.where("%s_language not in ('zh','zh_Hant','ko','en','ja')" % words[-1])
                else:
                    if result2[0][item] is not None:
                        validate_df = validate_df.where(result1_df[item] == result2[0][item])
                    else:
                        validate_df = validate_df.where(result1_df[item].isNull())
            c = validate_df.count()
            print c,result1,result2
            if c>0:
                print 'pass'
            else:
                print 'failed!!!',result1, result2, x
        finally:
            result1_df.unpersist()
finally:
    bulk_df.unpersist()


In [0]:

from pyspark.sql.functions import count
from aadatapipelinecore.core.fs.device import s3
from aadatapipelinecore.core.fs import Conf
from aadatapipelinecore.core.log import logger
from requests.auth import HTTPBasicAuth
from aadatapipelinecore.core.utils.http import plain_get
import json

s3_bucket_list = s3.S3Bucket(Conf(bucket_name='b2c-prod-data-pipeline-unified-review'))
path_list = s3_bucket_list.all(prefix="unified/review_bulk_json_month.v3/", depth_is_1=True)
path_list = path_list[:102]
# print path_list


def get_path():
    index_list = []
    for path in path_list:
        # print path
        # total = spark.read.json("s3://b2c-prod-data-pipeline-unified-review/{}".format(path)).filter(
        #     "_identifier is not null").count()

        new_path = path.split("=")[1][:7].replace("-", "")

        index_list.append(("int-ss-review_v1-apple-store-ios-all-{}".format(new_path),
                           "int-ss-review_v1-google-play-android-all-{}".format(new_path)))
        # print new_path, total
    return index_list


def get_es_result(index):
    # print index
    headers = {
        'content-type': 'application/json'
    }
    auth = HTTPBasicAuth("bdp", "C38vEJEuraCw")
    url = "http://internal-internel-review-es-v7-1286999769.us-east-1.elb.amazonaws.com:19200/{}/_count".format(
        index)
    try:
        resp = plain_get(url, timeout=30, auth=auth, headers=headers, is_resource_of_appannie=False)
        doc = json.loads(resp.text)
        doc["count"]
    except KeyError:
        print index
        return 0

    return doc["count"]


def es_doc(index):
    # print  get_es_result(index[0])
    # print get_es_result(index[1])
    total_count = get_es_result(index[0]) + get_es_result(index[1])
    print 'total', total_count
    # if total_count != index[2]:
    print ' path : {}, {},  db count: {}'.format(index[0],index[1], total_count)
    # else:
    #     print index[0], total_count


def compare(index_list):
    for row in index_list[:30]:
        es_doc(row)
        
compare(get_path())

In [0]:

from aadatapipelinecore.core.log import logger

import unittest
from pyspark.sql.utils import AnalysisException
from aadatapipelinecore.core.fs.device import s3
from aadatapipelinecore.core.fs import Conf

source_path = 's3://b2c-prod-data-pipeline-unified-review/unified/review.v1/fact/process_granularity=monthly/'
dest_path = 's3://b2c-prod-data-pipeline-unified-review/unified/review_bulk_json_month.v3/date={}'

s3_bucket_list = s3.S3Bucket(Conf(bucket_name='b2c-prod-data-pipeline-unified-review'))
path_list = s3_bucket_list.all(prefix="unified/review.v1/fact/process_granularity=monthly/", depth_is_1=True)
print path_list[155]
for path in path_list[155:]:
    print path.split("/")[4].split("=")[1]

    count_1 = spark.read.parquet("s3://b2c-prod-data-pipeline-unified-review/{}".format(path)).where(
        "market_code in ('apple-store','google-play')").select("review_id").distinct().count()
    new_path = path.split("/")[4].split("=")[1]
    df = spark.read.json(dest_path.format(new_path)).cache()
    count_2 = df.filter("index is not null").count()
    count_3 = df.filter("_identifier is not null").count()

    # print count_1
    if count_1 == count_2 == count_3:
        pass
        # print 'pass', path
    else:
        print "not equal!!!!!!", count_1, count_2, count_3, path


In [0]:

from aadatapipelinecore.core.log import logger
import pyspark.sql.functions as F
import unittest
from pyspark.sql.utils import AnalysisException
from aadatapipelinecore.core.fs.device import s3
from aadatapipelinecore.core.fs import Conf


source_path='s3://b2c-prod-data-pipeline-unified-review/unified/review_load_temp/review_agg_v3/'
dest_path='s3://b2c-prod-data-pipeline-unified-review/unified/review_bulk_json.v3/date={}-01/'



s3_bucket_list = s3.S3Bucket(Conf(bucket_name='b2c-prod-data-pipeline-unified-review'))
path_list = s3_bucket_list.all(prefix="unified/review_load_temp/review_agg_v3/", depth_is_1=True)

# review_bulk_json_by_process_date.v3/date=2020-01-20
def compare_data(test_month):
    print test_month
    df1 = spark.read.parquet(
    "s3://b2c-prod-data-pipeline-unified-review/unified/review_load_temp/review_agg_v3/event_month={}".format(test_month))
    df2 = spark.read.json(
    "s3://b2c-prod-data-pipeline-unified-review/unified/review_bulk_json_by_process_date.v3/date={}-01".format(test_month)).withColumn("id_1",
                                                                                                         F.monotonically_increasing_id())

    sample_df = df2.sample(False, 0.001, seed=0).limit(5).take(5)
    # print sample_df


    review_id = ''
    remove_column = ['english_content', 'english_reply', 'english_title', 'chinese_content', 'chinese_reply',
                 'chinese_title', 'japanese_content', 'japanese_reply', 'japanese_title', 'korean_content',
                 'korean_reply', 'korean_title', 'other_language_content', 'other_language_reply',
                 'other_language_title', 'index', 'id_1'] #,'_corrupt_record', '_id','_index', 'routing']
    column = df2.columns

    for x in remove_column:
        column.remove(x)


    for sd in sample_df:
        id_1 = sd["id_1"]
        id_2 = ''
        # print id_1
        if id_1 % 2 == 0:
            id_2 = id_1 + 1
        else:
            id_2 = id_1
            id_1 = id_1 - 1

        df2.registerTempTable("result_detail_tmp")
        test_result_load = spark.sql("select * from result_detail_tmp where id_1 in ('{}', '{}' )".format(id_1, id_2)).take(
            2)
        review_id = test_result_load[0]["index"]["_id"]

        # print review_id


        df1.registerTempTable("result_tmp")

        test_result_transform = spark.sql(
            "select * from result_tmp WHERE review_id = '{}'".format(str(review_id))).take(1)

        for value in column:
            if value == 'date':
                if test_result_load[1][value] != test_result_transform[0]["time"].split("T")[0]:
                    print 'failed', test_result_load[1][value], test_result_transform[0][value]
            elif value == '_identifier':
                if str(test_result_load[1][value]) != str(test_result_transform[0][value]):
                    print 'failed', test_result_load[1][value], test_result_transform[0][value]

            else:
                if test_result_load[1][value] != test_result_transform[0][value]:
                    print 'failed', test_result_load[1][value], test_result_transform[0][value]


for x in path_list:
    compare_data(x.split("/")[3].split("=")[1])

In [0]:

from aadatapipelinecore.core.log import logger

import unittest
from pyspark.sql.utils import AnalysisException
from aadatapipelinecore.core.fs.device import s3
from aadatapipelinecore.core.fs import Conf

s3_bucket_list = s3.S3Bucket(Conf(bucket_name='b2c-prod-data-pipeline-unified-review'))
path_list = s3_bucket_list.all(prefix="unified/review_load_temp/review_load_temp_v4/", depth_is_1=True)
# print path_list
for path in path_list[:107]:
    count_1 = spark.read.parquet("s3://b2c-prod-data-pipeline-unified-review/{}".format(path)).select("review_id").distinct().count()
    count_2 = spark.read.parquet("s3://b2c-prod-data-pipeline-unified-review/unified/review_load_temp/review_agg_v3/{}".format(path.split("/")[3])).select("review_id").count()
    if count_1 != count_2:
        print "not equal!!!!!!", count_1 , count_2, path
    else:
        print 'pass', path

In [0]:

from pyspark.sql.functions import desc

#dup_list = spark.read.parquet('s3://b2c-prod-data-pipeline-unified-review/unified/review_load_temp/review_load_temp_v4/event_month=2019-09/').select('review_id','process_date','process_hour').distinct().groupBy("review_id").agg({"*":"count"}).where("count(1)>1").orderBy("count(1)",ascending = False).take(10)

for x in dup_list:
    result1 = spark.read.parquet('s3://b2c-prod-data-pipeline-unified-review/unified/review_load_temp/review_load_temp_v4/event_month=2019-09').filter("review_id='{}'".format(x["review_id"])).groupBy("process_date","process_hour","time").agg({"*":"count"}).orderBy(desc("time"),desc("process_date"),desc("process_hour")).take(1)
    result2 = spark.read.parquet("s3://b2c-prod-data-pipeline-unified-review/unified/review_load_temp/review_agg_v1/event_month=2019-09").filter("review_id='{}'".format(x["review_id"])).take(1)
    print result1,result2
    if result1[0]["process_date"]==result2[0]["process_date"] and result1[0]["process_hour"]==result2[0]["process_hour"] and result1[0]["time"]==result2[0]["time"]  :
        print 'pass'
    else:
        print 'failed!!!',result1, result2,x


In [0]:

from aadatapipelinecore.core.log import logger
import pyspark.sql.functions as F
import unittest
from pyspark.sql.utils import AnalysisException
from aadatapipelinecore.core.fs.device import s3
from aadatapipelinecore.core.fs import Conf


source_path='s3://b2c-prod-data-pipeline-unified-review/unified/review_load_temp/review_agg_v3/'
dest_path='s3://b2c-prod-data-pipeline-unified-review/unified/review_bulk_json.v3/date={}-01/'



s3_bucket_list = s3.S3Bucket(Conf(bucket_name='b2c-prod-data-pipeline-unified-review'))
path_list = s3_bucket_list.all(prefix="unified/review_load_temp/review_agg_v3/", depth_is_1=True)

    
    
    
def get_monthly_sample_data(monthly):
    
    review_df = spark.read.parquet("s3://b2c-prod-data-pipeline-unified-review/unified/review_load_temp/review_agg_v3/event_month={}/".format(monthly))
    review_df.registerTempTable("reviews")
    ios_samples_df = spark.sql("SELECT * FROM reviews WHERE market_code = 'apple-store' LIMIT 50").cache()
    ios_samples_df.count()
    android_samples_df = spark.sql("SELECT * FROM reviews WHERE market_code = 'google-play' LIMIT 50").cache()
    android_samples_df.count()
    samples = []
    month_index = monthly.replace("-","")
    for idx, df in [
        ("int-ss-review_v1-apple-store-ios-all-{}".format(month_index), ios_samples_df),
        ("int-ss-review_v1-google-play-android-all-{}".format(month_index), android_samples_df)]:
        print df.columns
        rows = (
            df
            .withColumn("date", F.date_format("time", "yyyy-MM-dd"))
            .drop("time")
            .withColumn("chinese_content", F.when((df.content_language == "zh") | (df.content_language == "zh_Hant"), df.content))
            .withColumn("chinese_reply", F.when((df.reply_language == "zh") | (df.reply_language == "zh_Hant"), df.reply))
            .withColumn("chinese_title", F.when((df.title_language == "zh") | (df.title_language == "zh_Hant"), df.title))
            .withColumn("english_content", F.when(df.content_language == "en", df.content))
            .withColumn("english_reply", F.when(df.reply_language == "en", df.reply))
            .withColumn("english_title", F.when(df.title_language == "en", df.title))
            .withColumn("japanese_content", F.when(df.content_language == "ja", df.content))
            .withColumn("japanese_reply", F.when(df.reply_language == "ja", df.reply))
            .withColumn("japanese_title", F.when(df.title_language == "ja", df.title))
            .withColumn("korean_content", F.when(df.content_language == "ko", df.content))
            .withColumn("korean_reply", F.when(df.reply_language == "ko", df.reply))
            .withColumn("korean_title", F.when(df.title_language == "ko", df.title))
            .withColumn("other_language_content", F.when(
                (df.content_language != "zh") & (df.content_language != "zh_Hant") & (df.content_language != "en") &
                (df.content_language != "ja") & (df.content_language != "ko"), df.content))
            .withColumn("other_language_reply", F.when(
                (df.reply_language != "zh") & (df.reply_language != "zh_Hant") & (df.reply_language != "en") &
                (df.reply_language != "ja") & (df.reply_language != "ko"), df.reply))
            .withColumn("other_language_title", F.when(
                (df.title_language != "zh") & (df.title_language != "zh_Hant") & (df.title_language != "en") &
                (df.title_language != "ja") & (df.title_language != "ko"), df.title))
            .collect()
        )
        samples.append((idx, rows))
    return samples
    
    
sample_data_set=[]
for x in path_list[100:]:
    sample_data_set.append(get_monthly_sample_data(x.split("/")[3].split("=")[1]))
    
def compare(doc, row):
    keys = [u"review_id", u'_identifier', u'app_id', u'chinese_content', u'chinese_reply', u'chinese_title', u'content', u'content_language', u'country_code', u'date', u'device_code', u'english_content', u'english_reply', u'english_title', u'japanese_content', u'japanese_reply', u'japanese_title', u'korean_content', u'korean_reply', u'korean_title', u'language', u'market_code', u'other_language_content', u'other_language_reply', u'other_language_title', u'product_version', u'rating', u'reply', u'reply_date', u'reply_language', u'title', u'title_language', u'user_device', u'user_id', u'user_language', u'user_name', u'user_purchased']
    row_dict =  row.asDict()
    for key in keys:
        if key in ("app_id", "_identifier"):
            assert str(doc[key]) == str(row_dict[key]), "{} is diffrent".format(key)
        else:
            assert doc[key] == row_dict[key], "{} is diffrent".format(key)
def es_doc(index, doc_id, routing_id):
    headers = {
        'content-type': 'application/json'
    }
    auth = HTTPBasicAuth("bdp","C38vEJEuraCw")
    
    url = "http://internal-internel-review-es-v7-1286999769.us-east-1.elb.amazonaws.com:19200/{}/_doc/{}?routing={}".format(index, doc_id, routing_id)
    resp = plain_get(url, timeout=30, auth=auth, headers=headers, is_resource_of_appannie=False)
    doc = json.loads(resp.text)
    source = doc['_source']
    source['review_id'] = doc["_id"]
    return source
    
    
for idx, rows in samples:
    for row in rows:
        doc = es_doc(idx, row.review_id, row.app_id)
        compare(doc, row)


In [0]:

import pyspark.sql.functions as F

# path = "s3://b2c-prod-data-pipeline-unified-review/unified/review.v1/fact/process_granularity={weekly,daily,hourly}/process_date=%s/" % x["process_date"].strftime("%Y-%m-%d")
# total_count = spark.read.option("basePath","s3://b2c-prod-data-pipeline-unified-review/unified/review.v1/fact/").parquet(path).where("market_code in ('apple-store','google-play')").count()

review_df = spark.read.parquet("s3://b2c-prod-data-pipeline-unified-review/unified/review_bulk_json.v1/date=2019-01-01/")
review_df.registerTempTable("reviews")
ios_samples_df = spark.sql("SELECT * FROM reviews WHERE market_code = 'apple-store' LIMIT 50").cache()
ios_samples_df.count()
android_samples_df = spark.sql("SELECT * FROM reviews WHERE market_code = 'google-play' LIMIT 50").cache()
android_samples_df.count()
samples = []
for idx, df in [
    ("int-ss-review_v1-apple-store-ios-all-200807", ios_samples_df), 
    ("int-ss-review_v1-google-play-android-all-200807", android_samples_df)]:
    print df.columns
    rows = (
        df
        .withColumn("date", F.date_format("time", "yyyy-MM-dd"))
        .drop("time")
        .withColumn("chinese_content", F.when((df.content_language == "zh") | (df.content_language == "zh_Hant"), df.content))
        .withColumn("chinese_reply", F.when((df.reply_language == "zh") | (df.reply_language == "zh_Hant"), df.reply))
        .withColumn("chinese_title", F.when((df.title_language == "zh") | (df.title_language == "zh_Hant"), df.title))
        .withColumn("english_content", F.when(df.content_language == "en", df.content))
        .withColumn("english_reply", F.when(df.reply_language == "en", df.reply))
        .withColumn("english_title", F.when(df.title_language == "en", df.title))
        .withColumn("japanese_content", F.when(df.content_language == "ja", df.content))
        .withColumn("japanese_reply", F.when(df.reply_language == "ja", df.reply))
        .withColumn("japanese_title", F.when(df.title_language == "ja", df.title))
        .withColumn("korean_content", F.when(df.content_language == "ko", df.content))
        .withColumn("korean_reply", F.when(df.reply_language == "ko", df.reply))
        .withColumn("korean_title", F.when(df.title_language == "ko", df.title))
        .withColumn("other_language_content", F.when(
            (df.content_language != "zh") & (df.content_language != "zh_Hant") & (df.content_language != "en") &
            (df.content_language != "ja") & (df.content_language != "ko"), df.content))
        .withColumn("other_language_reply", F.when(
            (df.reply_language != "zh") & (df.reply_language != "zh_Hant") & (df.reply_language != "en") &
            (df.reply_language != "ja") & (df.reply_language != "ko"), df.reply))
        .withColumn("other_language_title", F.when(
            (df.title_language != "zh") & (df.title_language != "zh_Hant") & (df.title_language != "en") &
            (df.title_language != "ja") & (df.title_language != "ko"), df.title))
        .collect()
    )
    samples.append((idx, rows))
print samples


In [0]:

import json
import time
from requests.auth import HTTPBasicAuth
from aadatapipelinecore.core.utils.http import plain_get
def compare(doc, row):
    keys = [u"review_id", u'_identifier', u'app_id', u'chinese_content', u'chinese_reply', u'chinese_title', u'content', u'content_language', u'country_code', u'date', u'device_code', u'english_content', u'english_reply', u'english_title', u'japanese_content', u'japanese_reply', u'japanese_title', u'korean_content', u'korean_reply', u'korean_title', u'language', u'market_code', u'other_language_content', u'other_language_reply', u'other_language_title', u'product_version', u'rating', u'reply', u'reply_date', u'reply_language', u'title', u'title_language', u'user_device', u'user_id', u'user_language', u'user_name', u'user_purchased']
    row_dict =  row.asDict()
    for key in keys:
        if key in ("app_id", "_identifier"):
            assert str(doc[key]) == str(row_dict[key]), "{} is diffrent".format(key)
        else:
            assert doc[key] == row_dict[key], "{} is diffrent".format(key)
def es_doc(index, doc_id, routing_id):
    headers = {
        'content-type': 'application/json'
    }
    auth = HTTPBasicAuth("bdp","C38vEJEuraCw")
    
    url = "http://internal-internel-review-es-v7-1286999769.us-east-1.elb.amazonaws.com:19200/{}/_doc/{}?routing={}".format(index, doc_id, routing_id)
    resp = plain_get(url, timeout=30, auth=auth, headers=headers, is_resource_of_appannie=False)
    doc = json.loads(resp.text)
    source = doc['_source']
    source['review_id'] = doc["_id"]
    return source
for idx, rows in samples:
    for row in rows:
        doc = es_doc(idx, row.review_id, row.app_id)
        compare(doc, row)




In [0]:

from dateutil.relativedelta import relativedelta
import datetime
from pyspark.sql.utils import AnalysisException
def generate_data():
    result = []
    start_date = datetime.date(2018, 8, 14) 
    today = datetime.date(2019, 12, 31)    
    while start_date < today:
        delta_date=relativedelta(days=1)
        start_date = start_date + delta_date
        result.append((start_date))
    return result

print generate_data()
for x in generate_data():
    cal_date=x.strftime("%Y-%m-%d")
    # print("time:", cal_date)
    spark_result=''
    print "s3://prod_appannie_uniform_public_app_store_data/review-uniform/DAILY/{process_date}/23/".format(process_date=cal_date)
    try:
        spark_result=spark.read.csv("s3://prod_appannie_uniform_public_app_store_data/review-uniform/DAILY/{process_date}/23/".format(process_date=cal_date),sep="\t").filter("_c6 is not null").take(2)
    except AnalysisException:
        pass
    if spark_result:
        print spark_result
    # if result:
    #     print result



In [0]:

from pyspark.sql.functions import count, avg
df = spark.read.csv("s3://prod_appannie_uniform_public_app_store_data/review-uniform/HOURLY/2020-01-15/05/gp/",sep="\t").selectExpr("_c0 as id", "_c1 as product_id", "_c2 as product_version", "_c3 as user_id", "_c4 as user_name" , "_c5 as user_language", "_c6 as user_device", "_c7 as user_purchased", "_c8 as rating", "_c9 as country", "_c10 as date","_c11 as language", "_c12 as title", "_c13 as title_t", "_c14 as content", "_c15 as content_t", "_c16 as reply", "_c17 as reply_t", "_c18 as reply_date")
result= df.groupBy("id").agg(count("id")).orderBy("count(id)", ascending=False).take(20)

for x in result:
    print x

In [0]:


from aadatapipelinecore.core.log import logger
import unittest
from pyspark.sql.utils import AnalysisException
from aadatapipelinecore.core.fs.device import s3
from aadatapipelinecore.core.fs import Conf
dest_path='s3://b2c-prod-data-pipeline-unified-review/unified/review_bulk_json_by_process_date.v3/date={}/'
s3_bucket_list = s3.S3Bucket(Conf(bucket_name='b2c-prod-data-pipeline-unified-review'))
path_list = ['process_granularity={hourly,daily,weekly}/process_date=%s/' % item for item in ["2020-02-24"]]
print path_list[0]
for path in path_list:
    print path.split("/")[1].split("=")[1]
    count_1 = spark.read.option('basePath','s3://b2c-prod-data-pipeline-unified-review/unified/review.v1/fact/').parquet("s3://b2c-prod-data-pipeline-unified-review/unified/review.v1/fact/{}".format(path)).where("market_code in ('apple-store','google-play') and device_code in ('android-all', 'ios-all')").select("review_id").distinct().count()
    df = spark.read.json(dest_path.format(path.split("/")[1].split("=")[1])).cache()
    count_2 = df.filter("index is not null").count()
    count_3 = df.filter("_identifier is not null").count()
    print count_1, count_2, count_3
    if count_1 == count_2 == count_3 :
        print 'pass', path
    else:
        print "not equal!!!!!!", count_1 , count_2, count_3,  path


In [0]:

import unittest
import boto3
from pyspark.sql.utils import AnalysisException

platforms = {
    'amazon': {"device_code": "android-all", "market_code": "amazon-store"},
    'gp': {"device_code": "android-all", "market_code": "google-play"},
    'ios': {"device_code": "ios-all", "market_code": "apple-store"},
    'win': {"device_code": "all", "market_code": "windows-store"},
    'wp': {"device_code": "windows-phone", "market_code": "windows-store"}
}

s3 = boto3.resource('s3')
my_bucket = s3.Bucket('prod_appannie_uniform_public_app_store_data')


# get current path list
path=[]
for object_summary in my_bucket.objects.filter(Prefix="review-uniform/MONTHLY/"):
    path.append(object_summary.key[:object_summary.key.rfind("/")])

path = list(dict.fromkeys(path))
# print path

# compared the data from uniform(raw data) to unified data
raw_path="s3://prod_appannie_uniform_public_app_store_data/"
unified_path="s3://b2c-prod-data-pipeline-unified-review/unified/review.v1/fact/process_granularity=monthly/process_date={unified_process_date}/process_hour=23/device_code={unified_device_code}/market_code={unified_market_code}"

def count_raw_unified_data():
    t = unittest.TestCase('run')
    for p in path:
        raw_count = spark.read.csv(raw_path+p,sep="\t").count()
        # print raw_path+p
        # print 'raw_count', raw_count
        if raw_count == 0:
            # print 'raw is empty, path is : ' , raw_path+p
            continue
        
        upath=unified_path.format(unified_process_date=p.split("/")[2],unified_device_code=platforms[p.split("/")[3]]["device_code"],unified_market_code=platforms[p.split("/")[3]]["market_code"] )
        try:
            unified_count = spark.read.parquet(upath).count()
        except AnalysisException as e:
            print 'the path is not exist for unified level!!! the path is', raw_path+p, upath
        
        # print 'unified_count', unified_count

        if raw_count != unified_count:
            print 'count is not equal!!! the path is : ', raw_path+p, upath, "raw is: {} , unified is : {} ".format(raw_count, unified_count)



count_raw_unified_data()


In [0]:

import unittest
import boto3
from pyspark.sql.utils import AnalysisException
from aadatapipelinecore.core.fs.device import s3 
from aadatapipelinecore.core.fs import Conf

platforms = {
    'amazon': {"device_code": "android-all", "market_code": "amazon-store"},
    'gp': {"device_code": "android-all", "market_code": "google-play"},
    'ios': {"device_code": "ios-all", "market_code": "apple-store"},
    'win': {"device_code": "all", "market_code": "windows-store"},
    'wp': {"device_code": "windows-phone", "market_code": "windows-store"}
}


# get up level data set
s3_bucket_list = s3.S3Bucket(Conf(bucket_name='prod_appannie_uniform_public_app_store_data'))
path_w= s3_bucket_list.all(prefix="review-uniform/WEEKLY/", depth_is_1=True )
path_w = path_w[10:]


# compared the data from uniform(raw data) to unified data
raw_path_weekly="s3://prod_appannie_uniform_public_app_store_data/"
unified_path_weekly="s3://b2c-prod-data-pipeline-unified-review/unified/review.v1/fact/process_granularity=weekly/process_date={unified_process_date}/process_hour=23/device_code={unified_device_code}/market_code={unified_market_code}"

def count_raw_unified_data_weekly():
    t = unittest.TestCase('run')
    for p in path_w:
        deep_raw_path=s3_bucket_list.all(p+'23', depth_is_1=True)
        for dp in deep_raw_path:
            # print 'raw_path is: ', raw_path_weekly+dp
            raw_count = spark.read.csv(raw_path_weekly+dp,sep="\t").count()
       
            if raw_count == 0:
                continue
        
            upath=unified_path_weekly.format(unified_process_date=dp.split("/")[2],unified_device_code=platforms[dp.split("/")[4]]["device_code"],unified_market_code=platforms[dp.split("/")[4]]["market_code"] )
            try:
                unified_count = spark.read.parquet(upath).count()
            except AnalysisException as e:
                print 'the path is not exist for unified level!!! the path is', raw_path+dp, upath
        
    
            if raw_count != unified_count:
                print 'count is not equal!!! the path is : ', raw_path+dp, upath, "raw is: {} , unified is : {} ".format(raw_count, unified_count)



count_raw_unified_data_weekly()


In [0]:

from aadatapipelinecore.core.log import logger

import unittest
from pyspark.sql.utils import AnalysisException
from aadatapipelinecore.core.fs.device import s3
from aadatapipelinecore.core.fs import Conf

s3_bucket_list = s3.S3Bucket(Conf(bucket_name='prod_appannie_uniform_public_app_store_data'))
path_d = s3_bucket_list.all(prefix="review-uniform/HOURLY/", depth_is_1=True)
# print path_d
print path_d[274]

platforms = {
    'amazon': {"device_code": "android-all", "market_code": "amazon-store"},
    'gp': {"device_code": "android-all", "market_code": "google-play"},
    'ios': {"device_code": "ios-all", "market_code": "apple-store"},
    'win': {"device_code": "all", "market_code": "windows-store"},
    'wp': {"device_code": "windows-phone", "market_code": "windows-store"}
}

# compared the data from uniform(raw data) to unified data
raw_path_d = "s3://prod_appannie_uniform_public_app_store_data/"
unified_path_d = "s3://b2c-prod-data-pipeline-unified-review/unified/review.v1/fact/process_granularity=hourly/process_date={unified_process_date}/process_hour={unified_process_hour}/device_code={unified_device_code}/market_code={unified_market_code}"


def count_raw_unified_data_hourly():
    unified_count = 0
    for p in path_d[985:]:
        process_hour_raw_path = s3_bucket_list.all(p, depth_is_1=True)
        for x in process_hour_raw_path:
            logger.info('raw path with process hour : '.format(x))
            platforms_process_hour_raw_path_list = s3_bucket_list.all(x, depth_is_1=True)

            # raw_count_sum, root_raw_data_path = calculate_sum_data_hourly(platforms_process_hour_raw_path_list)
            # logger.info('raw_count_sum : '.format(raw_count_sum))
            for dp in platforms_process_hour_raw_path_list:
                print dp
                root_raw_data_path = dp
                raw_count_hourly = spark.read.csv(raw_path_d + dp, sep="\t").count()
                logger.info('raw count : {}'.format(raw_count_hourly))

                if raw_count_hourly == 0:
                    continue
                upath = unified_path_d.format(unified_process_date=root_raw_data_path.split("/")[2],
                                          unified_device_code=platforms[root_raw_data_path.split("/")[4]][
                                              "device_code"],
                                          unified_market_code=platforms[root_raw_data_path.split("/")[4]][
                                              "market_code"],
                                          unified_process_hour=str(int(root_raw_data_path.split("/")[3])))
                try:
                    unified_count = spark.read.parquet(upath).count()
                    logger.info('unified_count : {}'.format(unified_count))

                except AnalysisException as e:
                    logger.info('the path is not exist for unified level!!! the raw path is {}, unified path is {}'.format(
                    raw_path_d + root_raw_data_path, upath))

            # logger.info( 'unified_count', unified_count

                if raw_count_hourly != unified_count:
                    logger.info(
                    'count is not equal!!! the raw path is {}, unified path is {} , raw count is : {}, unified count is : {}'.format(
                        raw_path_d + root_raw_data_path, upath,
                        raw_count_hourly, unified_count))


def calculate_sum_data_hourly( deep_raw_path):
    logger.info(deep_raw_path)
    count_one_day = 0
    for dp in deep_raw_path:
        raw_count = spark.read.csv(raw_path_d + dp, sep="\t").count()
        count_one_day = count_one_day + raw_count

    return count_one_day, deep_raw_path[0]

# count_raw_unified_data_hourly()


In [0]:

from aadatapipelinecore.core.log import logger
import time
import unittest
from pyspark.sql.utils import AnalysisException
from aadatapipelinecore.core.fs.device import s3
from aadatapipelinecore.core.fs import Conf
raw_path_d = "s3://prod_appannie_uniform_public_app_store_data/"

s3_bucket_list = s3.S3Bucket(Conf(bucket_name='prod_appannie_uniform_public_app_store_data'))
path_daily_count = s3_bucket_list.list(prefix="review-uniform/DAILY/2017-06-11/23/", limit=100, depth_is_1=False)

path_daily_count=list(map("s3://prod_appannie_uniform_public_app_store_data/".__add__,path_daily_count))

print path_daily_count

total_count=0
start=time.time()
total_count = spark.read.csv(path_daily_count).count()
end=time.time()
print end-start
print total_count
# print spark.read.csv("s3://prod_appannie_uniform_public_app_store_data/review-uniform/DAILY/2017-06-11/23/gp").count()
# print spark.read.parquet("s3://b2c-prod-data-pipeline-unified-review/unified/review.v1/fact/process_granularity=daily/process_date=2017-06-10/process_hour=23/device_code=android-all/market_code=google-play").count()

In [0]:

from aadatapipelinecore.core.log import logger
from aadatapipelinecore.core.fs.device import s3
from aadatapipelinecore.core.fs import Conf



platforms = {
    'amazon': {"device_code": "android-all", "market_code": "amazon-store"},
    'gp': {"device_code": "android-all", "market_code": "google-play"},
    'ios': {"device_code": "ios-all", "market_code": "apple-store"},
    'win': {"device_code": "all", "market_code": "windows-store"},
    'wp': {"device_code": "windows-phone", "market_code": "windows-store"}
}

# compared the data from uniform(raw data) to unified data
raw_path = "s3://prod_appannie_uniform_public_app_store_data/"
unified_path = "s3://b2c-prod-data-pipeline-unified-review/unified/review.v1/fact/process_granularity=daily/process_date={unified_process_date}/process_hour=23/device_code={unified_device_code}/market_code={unified_market_code}"

s3_bucket_list = s3.S3Bucket(Conf(bucket_name='prod_appannie_uniform_public_app_store_data'))
path_h = s3_bucket_list.all(prefix="review-uniform/MONTHLY/", depth_is_1=True)


def get_data_frame(column_length, data_path):
    if len(column_length) == 19:
        # print 'this is 19'
        raw_result = spark.read.csv(data_path, sep="\t").selectExpr("_c0 as id", "_c1 as product_id",
                                                                           "_c2 as product_version", "_c3 as user_id",
                                                                           "_c4 as user_name", "_c5 as user_language",
                                                                           "_c6 as user_device", "_c7 as user_purchased",
                                                                           "_c8 as rating", "_c9 as country",
                                                                           "_c10 as date", "_c11 as language",
                                                                           "_c12 as title", "_c13 as title_t",
                                                                           "_c14 as content", "_c15 as content_t",
                                                                           "_c16 as reply", "_c17 as reply_t",
                                                                           "_c18 as reply_date").take(1)
        column_dic = {"product_id": "app_id", "product_version": "product_version", "user_id": "user_id",
                      "user_name": "user_name", "user_language": "user_language", "user_device": "user_device",
                      "user_purchased": "user_purchased", "rating": "rating", "country": "country_code", "date": "time",
                      "title": "title", "content": "content", "reply": "reply", "reply_date": "reply_date"
                      }
        return raw_result,column_dic
    elif len(column_length) == 21:
        raw_result = spark.read.csv(data_path, sep="\t").selectExpr("_c0 as id", "_c1 as product_id", "_c2 as product_version", "_c3 as user_id", "_c4 as user_name" , "_c5 as user_language", "_c6 as user_device", "_c7 as user_purchased", "_c8 as userreview_url", "_c9 as rating", "_c10 as country","_c11 as date", "_c12 as language", "_c13 as title", "_c14 as title_t", "_c15 as content", "_c16 as content_t", "_c17 as reply_id", "_c18 as reply", "_c19 as reply_t", "_c20 as reply_date").take(1)
        column_dic = {"product_id": "app_id", "product_version": "product_version", "user_id": "user_id",
                      "user_name": "user_name", "user_language": "user_language", "user_device": "user_device",
                      "user_purchased": "user_purchased", "rating": "rating", "country": "country_code", "date": "time",
                      "title": "title", "content": "content", "reply": "reply", "reply_date": "reply_date", "reply_id":"reply_id", "userreview_url":"userreview_url"
                      }
        return raw_result,column_dic
    elif len(column_length) == 0:
        logger.info("data is empty")
        return [],{}
        
    
    
    
def check_raw_unified_data_equals():
    for p in path_h:
        sub_raw_path = s3_bucket_list.all(p, depth_is_1=True)
        for dp in sub_raw_path:
            print raw_path + dp
            global raw_result 
            raw_result = spark.read.csv(raw_path + dp, sep="\t").cache().take(1)
            
            if len(raw_result) != 0:
                # print raw_result[0]
                sample_raw_result, column_dic = get_data_frame(raw_result[0], raw_path + dp)
                # print sample_raw_result, column_dic
                if sample_raw_result:
                    check_id = sample_raw_result[0]["id"]
                    # print dp
                    upath = unified_path.format(unified_process_date=dp.split("/")[2],
                                        unified_device_code=platforms[dp.split("/")[4]]["device_code"],
                                        unified_market_code=platforms[dp.split("/")[4]]["market_code"])

                    unified_result = spark.read.parquet(upath).filter("review_id='{test_id}'".format(test_id=check_id)).take(2)

                   

                    for key, value in column_dic.items():
                        check_value(key, sample_raw_result[0][key], unified_result[0][value], p)

            elif len(raw_result) == 0:
                print 'No data'
                continue


def check_value(index, exp, act, p):
    # print index, exp, act, p
    try:
        if exp:
            if index == 'rating' or index == "product_id":
                exp = int(exp)
                act = int(act)
                if exp == act:
                    pass
                elif exp != act:
                    logger.info( 'number NOT Equal!!! !!!!!!!! index is {} , path is: {}  value is {} ~ {}'.format(index, p, exp,
                                                                                                        act))
            elif index == 'user_purchased':
                exp = str(exp).lower()
                act = str(act).lower()
                if exp == act:
                    pass
                if exp != act:
                    logger.info( 'NOT Equal!!! !!!!!!!! index is {} , path is: {}  value is {} ~ {}'.format(index, p, exp, act))
            else:
                if exp.lower().strip('"') == act.lower().strip('"'):
                    pass
                if exp != act:
                    logger.info( 'NOT Equal!!! !!!!!!!! index is {} , path is: {}  value is {} ~ {}'.format(index, p, exp, act))

    except UnicodeEncodeError as e:
        logger.info(e)
        
        


    
    
    
check_raw_unified_data_equals()


In [0]:

from aadatapipelinecore.core.log import logger
from aadatapipelinecore.core.fs.device import s3
from aadatapipelinecore.core.fs import Conf



platforms = {
    'amazon': {"device_code": "android-all", "market_code": "amazon-store"},
    'gp': {"device_code": "android-all", "market_code": "google-play"},
    'ios': {"device_code": "ios-all", "market_code": "apple-store"},
    'win': {"device_code": "all", "market_code": "windows-store"},
    'wp': {"device_code": "windows-phone", "market_code": "windows-store"}
}

# compared the data from uniform(raw data) to unified data
raw_path = "s3://prod_appannie_uniform_public_app_store_data/"
unified_path = "s3://b2c-prod-data-pipeline-unified-review/unified/review.v1/fact/process_granularity=weekly/process_date={unified_process_date}/process_hour={unified_process_hour}/device_code={unified_device_code}/market_code={unified_market_code}"

s3_bucket_list = s3.S3Bucket(Conf(bucket_name='prod_appannie_uniform_public_app_store_data'))
path_w = s3_bucket_list.all(prefix="review-uniform/WEEKLY/", depth_is_1=True)
path_w = path_w[14:]

def get_data_frame(column_length, data_path):
    if len(column_length) == 19:
        # print 'this is 19'
        raw_result = spark.read.csv(data_path, sep="\t").selectExpr("_c0 as id", "_c1 as product_id",
                                                                           "_c2 as product_version", "_c3 as user_id",
                                                                           "_c4 as user_name", "_c5 as user_language",
                                                                           "_c6 as user_device", "_c7 as user_purchased",
                                                                           "_c8 as rating", "_c9 as country",
                                                                           "_c10 as date", "_c11 as language",
                                                                           "_c12 as title", "_c13 as title_t",
                                                                           "_c14 as content", "_c15 as content_t",
                                                                           "_c16 as reply", "_c17 as reply_t",
                                                                           "_c18 as reply_date").take(1)
        column_dic = {"product_id": "app_id", "product_version": "product_version", "user_id": "user_id",
                      "user_name": "user_name", "user_language": "user_language", "user_device": "user_device",
                      "user_purchased": "user_purchased", "rating": "rating", "country": "country_code", "date": "time",
                      "title": "title", "content": "content", "reply": "reply", "reply_date": "reply_date"
                      }
        return raw_result,column_dic
    elif len(column_length) == 21:
        raw_result = spark.read.csv(data_path, sep="\t").selectExpr("_c0 as id", "_c1 as product_id", "_c2 as product_version", "_c3 as user_id", "_c4 as user_name" , "_c5 as user_language", "_c6 as user_device", "_c7 as user_purchased", "_c8 as userreview_url", "_c9 as rating", "_c10 as country","_c11 as date", "_c12 as language", "_c13 as title", "_c14 as title_t", "_c15 as content", "_c16 as content_t", "_c17 as reply_id", "_c18 as reply", "_c19 as reply_t", "_c20 as reply_date").take(1)
        column_dic = {"product_id": "app_id", "product_version": "product_version", "user_id": "user_id",
                      "user_name": "user_name", "user_language": "user_language", "user_device": "user_device",
                      "user_purchased": "user_purchased", "rating": "rating", "country": "country_code", "date": "time",
                      "title": "title", "content": "content", "reply": "reply", "reply_date": "reply_date", "reply_id":"reply_id", "userreview_url":"userreview_url"
                      }
        return raw_result,column_dic
    elif len(column_length) == 0:
        logger.info("data is empty")
        return [],{}
        
    
    
    
def check_raw_unified_data_equals():
            # for dp in platforms_process_hour_raw_path_list:
            #     print dp
            #     root_raw_data_path = dp
            #     raw_count_hourly = spark.read.csv(raw_path_d + dp, sep="\t").count()
            #     logger.info('raw count : {}'.format(raw_count_hourly))
    for p in path_w:
        sub_raw_path = s3_bucket_list.all(p, depth_is_1=True)
        for sub_path in sub_raw_path:
            sub_path_list = s3_bucket_list.all(sub_path, depth_is_1=True)
            for dp in sub_path_list:
                print 'dp_path is :' , raw_path + dp

                global raw_result 
                raw_result = spark.read.csv(raw_path + dp, sep="\t").take(1)
            
                if len(raw_result) != 0:
                    # print raw_result[0]
                    sample_raw_result, column_dic = get_data_frame(raw_result[0], raw_path + dp)
                    # print sample_raw_result, column_dic
                    if sample_raw_result:
                        check_id = sample_raw_result[0]["id"]
                        # print dp
                        upath = unified_path.format(unified_process_date=dp.split("/")[2],
                                        unified_device_code=platforms[dp.split("/")[4]]["device_code"],
                                        unified_market_code=platforms[dp.split("/")[4]]["market_code"],
                                        unified_process_hour=dp.split("/")[3])

                        unified_result = spark.read.parquet(upath).filter("review_id='{test_id}'".format(test_id=check_id)).take(2)

                   

                        for key, value in column_dic.items():
                            check_value(key, sample_raw_result[0][key], unified_result[0][value], p)

                elif len(raw_result) == 0:
                    print 'No data'
                    continue


def check_value(index, exp, act, p):
    print index, exp, act, p
    try:
        if exp:
            if index == 'rating' or index == "product_id":
                exp = int(exp)
                act = int(act)
                if exp == act:
                    pass
                elif exp != act:
                    logger.info( 'number NOT Equal!!! !!!!!!!! index is {} , path is: {}  value is {} ~ {}'.format(index, p, exp,
                                                                                                        act))
            elif index == 'user_purchased':
                exp = str(exp).lower()
                act = str(act).lower()
                if exp == act:
                    pass
                if exp != act:
                    logger.info( 'NOT Equal!!! !!!!!!!! index is {} , path is: {}  value is {} ~ {}'.format(index, p, exp, act))
            else:
                if exp.lower().strip('"') == act.lower().strip('"'):
                    pass
                if exp != act:
                    logger.info( 'NOT Equal!!! !!!!!!!! index is {} , path is: {}  value is {} ~ {}'.format(index, p, exp, act))

    except UnicodeEncodeError as e:
        logger.info(e)
        
        


    
    
    
check_raw_unified_data_equals()


In [0]:

from aadatapipelinecore.core.log import logger
from aadatapipelinecore.core.fs.device import s3
from aadatapipelinecore.core.fs import Conf
from pyspark.sql.utils import AnalysisException
import random

platforms = {
    'amazon': {"device_code": "android-all", "market_code": "amazon-store"},
    'gp': {"device_code": "android-all", "market_code": "google-play"},
    'ios': {"device_code": "ios-all", "market_code": "apple-store"},
    'win': {"device_code": "all", "market_code": "windows-store"},
    'wp': {"device_code": "windows-phone", "market_code": "windows-store"}
}


def get_data_frame(spark, column_length, data_path):
    if len(column_length) == 19:
        raw_result = spark.read.csv(data_path, sep="\t").sample(False, 0.1, seed=0).limit(1).selectExpr("_c0 as id",
                                                                                                        "_c1 as product_id",
                                                                                                        "_c2 as product_version",
                                                                                                        "_c3 as user_id",
                                                                                                        "_c4 as user_name",
                                                                                                        "_c5 as user_language",
                                                                                                        "_c6 as user_device",
                                                                                                        "_c7 as user_purchased",
                                                                                                        "_c8 as rating",
                                                                                                        "_c9 as country",
                                                                                                        "_c10 as date",
                                                                                                        "_c11 as language",
                                                                                                        "_c12 as title",
                                                                                                        "_c13 as title_t",
                                                                                                        "_c14 as content",
                                                                                                        "_c15 as content_t",
                                                                                                        "_c16 as reply",
                                                                                                        "_c17 as reply_t",
                                                                                                        "_c18 as reply_date").take(
            1)
        column_dic = {"product_id": "app_id", "product_version": "product_version", "user_id": "user_id",
                      "user_name": "user_name", "user_language": "user_language", "user_device": "user_device",
                      "user_purchased": "user_purchased", "rating": "rating", "country": "country_code", "date": "time",
                      "title": "title", "content": "content", "reply": "reply", "reply_date": "reply_date"
                      }
        return raw_result, column_dic
    elif len(column_length) == 21:
        raw_result = spark.read.csv(data_path, sep="\t").sample(False, 0.1, seed=0).limit(1).selectExpr("_c0 as id",
                                                                                                        "_c1 as product_id",
                                                                                                        "_c2 as product_version",
                                                                                                        "_c3 as user_id",
                                                                                                        "_c4 as user_name",
                                                                                                        "_c5 as user_language",
                                                                                                        "_c6 as user_device",
                                                                                                        "_c7 as user_purchased",
                                                                                                        "_c8 as userreview_url",
                                                                                                        "_c9 as rating",
                                                                                                        "_c10 as country",
                                                                                                        "_c11 as date",
                                                                                                        "_c12 as language",
                                                                                                        "_c13 as title",
                                                                                                        "_c14 as title_t",
                                                                                                        "_c15 as content",
                                                                                                        "_c16 as content_t",
                                                                                                        "_c17 as reply_id",
                                                                                                        "_c18 as reply",
                                                                                                        "_c19 as reply_t",
                                                                                                        "_c20 as reply_date").take(
            1)
        column_dic = {"product_id": "app_id", "product_version": "product_version", "user_id": "user_id",
                      "user_name": "user_name", "user_language": "user_language", "user_device": "user_device",
                      "user_purchased": "user_purchased", "rating": "rating", "country": "country_code", "date": "time",
                      "title": "title", "content": "content", "reply": "reply", "reply_date": "reply_date",
                      "reply_id": "reply_id", "userreview_url": "userreview_url"
                      }
        return raw_result, column_dic
    elif len(column_length) == 0:
        logger.info("data is empty")
        return [], {}


def check_raw_unified_data_equals_with_granularity_for_one_day(granularity):
    raw_path_granularity = "s3://prod_appannie_uniform_public_app_store_data/"
    unified_path_granularity = "s3://b2c-prod-data-pipeline-unified-review/unified/review.v1/fact/process_granularity={unified_granularity}/process_date={unified_process_date}/process_hour={unified_process_hour}/device_code={unified_device_code}/market_code={unified_market_code}"

    s3_bucket_list = s3.S3Bucket(Conf(bucket_name='prod_appannie_uniform_public_app_store_data'))
    path_w = s3_bucket_list.all(prefix="review-uniform/{}/".format(granularity.upper()), depth_is_1=True)
    path_w = path_w[8:]
    random_path = random.sample(path_w, k=10)
    logger.info("random_path is {}".format(random_path))
    for p in random_path:
        sub_raw_path = s3_bucket_list.all(p, depth_is_1=True)
        for sub_path in sub_raw_path:
            sub_path_list = s3_bucket_list.all(sub_path, depth_is_1=True)
            for dp in sub_path_list:
                logger.info('dp_path is : {}'.format(raw_path_granularity + dp))
                global raw_result
                raw_result_list = spark.read.csv(raw_path_granularity + dp, sep="\t").collect()
                for raw_result in raw_result_list:
                    if len(raw_result) != 0:
                        # print raw_result[0]
                        sample_raw_result, column_dic = get_data_frame(raw_result[0], raw_path_granularity + dp)
                        # print sample_raw_result, column_dic
                        if sample_raw_result:
                            check_time = sample_raw_result[0]["time"]
                            # print dp
                            upath = unified_path_granularity.format(unified_process_date=dp.split("/")[2],
                                                                    unified_device_code=platforms[dp.split("/")[4]][
                                                                        "device_code"],
                                                                    unified_market_code=platforms[dp.split("/")[4]][
                                                                        "market_code"],
                                                                    unified_process_hour=dp.split("/")[3],
                                                                    unified_granularity=granularity)
                            try:
        
                                unified_result = spark.read.parquet(upath).filter("").take(2)
                            except AnalysisException as e:
                                logger.info(
                                    'the path is not exist for unified level!!! expected unified path is {}'.format(
                                        upath))
        
                            for key, value in column_dic.items():
                                check_value(key, sample_raw_result[0][key], unified_result[0][value], p)
        
                    elif len(raw_result) == 0:
                        print 'No data'
                        continue


def check_value(index, exp, act, p):
    logger.info(index, exp, act, p)
    try:
        if exp:
            if index == 'rating' or index == "product_id":
                exp = int(exp)
                act = int(act)
                if exp == act:
                    pass
                elif exp != act:
                    logger.info(
                        'number NOT Equal!!! !!!!!!!! index is {} , path is: {}  value is {} #~~~~# {}'.format(index, p,
                                                                                                               exp,
                                                                                                               act))
            elif index == 'user_purchased':
                exp = str(exp).lower()
                act = str(act).lower()
                if exp == act:
                    pass
                if exp != act:
                    logger.info(
                        'NOT Equal!!! !!!!!!!! index is {} , path is: {}  value is {} #~~~~# {}'.format(index, p, exp,
                                                                                                        act))
            else:
                if exp.lower().strip('"') == act.lower().strip('"'):
                    pass
                if exp != act:
                    logger.info(
                        'NOT Equal!!! !!!!!!!! index is {} , path is: {}  value is {} #~~~~# {}'.format(index, p, exp,
                                                                                                        act))

    except UnicodeEncodeError as e:
        logger.info(e)


# def main(spark, params):
#     # assert params.get('cluster_name'), _usage("cluster_name is required")
#     graularity = ["weekly", "daily", "hourly"]
#     for x in graularity:
#         logger.info("start to test granulairty as {}".format(x))
graularity = ["weekly", "daily", "hourly"]
for x in graularity:
    check_raw_unified_data_equals_with_granularity_for_one_day(x)
    logger.info("start to test granulairty as {}".format(x))



In [0]:
%python
import csv
from gzip import GzipFile
from StringIO import StringIO
TSV_TITLE_ORDER = [
    "id",
    "product_id",
    "product_version",
    "user_id",
    "user_name",
    "user_language",
    "user_device",
    "user_purchased",
    "userreview_url",
    "rating",
    "country",
    "date",
    "language",
    "title",
    "title_t",
    "content",
    "content_t",
    "reply_id",
    "reply",
    "reply_t",
    "reply_date"
]
f = open("/tmp/30600000000027.gz", "rb")
gz = GzipFile(None, 'rb', 9, StringIO(f.read()))
#print gz.read()
#gz = [gz.next()]
reader = csv.reader(gz, delimiter='\t', quotechar='"', quoting=csv.QUOTE_ALL)
tsv = [row for row in reader]
print "\n".join(("\t".join(item) for item in tsv))


In [0]:

from aadatapipelinecore.core.log import logger
from aadatapipelinecore.core.fs.device import s3
from aadatapipelinecore.core.fs import Conf

s3_bucket_list = s3.S3Bucket(Conf(bucket_name='prod_appannie_uniform_public_app_store_data'))
path_d = s3_bucket_list.all(prefix="review-uniform/DAILY/", depth_is_1=True)

date_list_raw = [ x.split("/")[2] for x in path_d ]


s3_bucket_list_unified = s3.S3Bucket(Conf(bucket_name='b2c-prod-data-pipeline-unified-review'))
path_d_unified = s3_bucket_list_unified.all(prefix="unified/review.v1/fact/process_granularity=daily/", depth_is_1=True)
date_list_unified = [ x.split("/")[4].split("=")[1] for x in path_d_unified ]

not_exist_data_set = sorted(set(date_list_raw)-set(date_list_unified))

print not_exist_data_set
# for x in not_exist_data_set:
#     el = "review-uniform//{}/".format(x)
#     path_d.remove(el)
# print path_d

In [0]:

from aadatapipelinecore.core.log import logger
from aadatapipelinecore.core.fs.device import s3
from aadatapipelinecore.core.fs import Conf
from pyspark.sql.utils import AnalysisException
import random

platforms = {
    'amazon': {"device_code": "android-all", "market_code": "amazon-store"},
    'gp': {"device_code": "android-all", "market_code": "google-play"},
    'ios': {"device_code": "ios-all", "market_code": "apple-store"},
    'win': {"device_code": "all", "market_code": "windows-store"},
    'wp': {"device_code": "windows-phone", "market_code": "windows-store"}
}


def get_data_frame(column_length, data_path):
    if len(column_length) == 19:
        raw_result = spark.read.csv(data_path, sep="\t").sample(False, 0.1, seed=0).limit(1).selectExpr("_c0 as id",
                                                                                                        "_c1 as product_id",
                                                                                                        "_c2 as product_version",
                                                                                                        "_c3 as user_id",
                                                                                                        "_c4 as user_name",
                                                                                                        "_c5 as user_language",
                                                                                                        "_c6 as user_device",
                                                                                                        "_c7 as user_purchased",
                                                                                                        "_c8 as rating",
                                                                                                        "_c9 as country",
                                                                                                        "_c10 as date",
                                                                                                        "_c11 as language",
                                                                                                        "_c12 as title",
                                                                                                        "_c13 as title_t",
                                                                                                        "_c14 as content",
                                                                                                        "_c15 as content_t",
                                                                                                        "_c16 as reply",
                                                                                                        "_c17 as reply_t",
                                                                                                        "_c18 as reply_date").take(
            1)
        column_dic = {"product_id": "app_id", "product_version": "product_version", "user_id": "user_id",
                      "user_name": "user_name", "user_language": "user_language", "user_device": "user_device",
                      "user_purchased": "user_purchased", "rating": "rating", "country": "country_code", "date": "time",
                      "title": "title", "content": "content", "reply": "reply", "reply_date": "reply_date"
                      }
        return raw_result, column_dic
    elif len(column_length) == 21:
        raw_result = spark.read.csv(data_path, sep="\t").sample(False, 0.1, seed=0).limit(1).selectExpr("_c0 as id",
                                                                                                        "_c1 as product_id",
                                                                                                        "_c2 as product_version",
                                                                                                        "_c3 as user_id",
                                                                                                        "_c4 as user_name",
                                                                                                        "_c5 as user_language",
                                                                                                        "_c6 as user_device",
                                                                                                        "_c7 as user_purchased",
                                                                                                        "_c8 as userreview_url",
                                                                                                        "_c9 as rating",
                                                                                                        "_c10 as country",
                                                                                                        "_c11 as date",
                                                                                                        "_c12 as language",
                                                                                                        "_c13 as title",
                                                                                                        "_c14 as title_t",
                                                                                                        "_c15 as content",
                                                                                                        "_c16 as content_t",
                                                                                                        "_c17 as reply_id",
                                                                                                        "_c18 as reply",
                                                                                                        "_c19 as reply_t",
                                                                                                        "_c20 as reply_date").take(
            1)
        column_dic = {"product_id": "app_id", "product_version": "product_version", "user_id": "user_id",
                      "user_name": "user_name", "user_language": "user_language", "user_device": "user_device",
                      "user_purchased": "user_purchased", "rating": "rating", "country": "country_code", "date": "time",
                      "title": "title", "content": "content", "reply": "reply", "reply_date": "reply_date",
                      "reply_id": "reply_id", "userreview_url": "userreview_url"
                      }
        return raw_result[0], column_dic
    elif len(column_length) == 0:
        logger.info("data is empty")
        return [], {}


def check_raw_unified_data_equals_with_granularity_for_one_day(granularity):
    raw_path_granularity = "s3://prod_appannie_uniform_public_app_store_data/"
    unified_path_granularity = "s3://b2c-prod-data-pipeline-unified-review/unified/review.v1/fact/process_granularity={unified_granularity}/process_date={unified_process_date}/process_hour={unified_process_hour}/device_code={unified_device_code}/market_code={unified_market_code}"

    s3_bucket_list = s3.S3Bucket(Conf(bucket_name='prod_appannie_uniform_public_app_store_data'))
    path_w = s3_bucket_list.all(prefix="review-uniform/{}/".format(granularity.upper()), depth_is_1=True)
    path_w = path_w[8:]
    random_path = random.sample(path_w, k=10)
    logger.info("random_path is {}".format(random_path))
    for p in random_path:
        sub_raw_path = s3_bucket_list.all(p, depth_is_1=True)
        for sub_path in sub_raw_path:
            sub_path_list = s3_bucket_list.all(sub_path, depth_is_1=True)
            for dp in sub_path_list:
                logger.info('dp_path is : {}'.format(raw_path_granularity + dp))
                global raw_result
                raw_result_list = spark.read.csv(raw_path_granularity + dp, sep="\t").collect()
                for raw_result in raw_result_list:
                    print raw_result
                    if len(raw_result) != 0:
                        # print raw_result[0]
                        sample_raw_result, column_dic = get_data_frame(raw_result, raw_path_granularity + dp)
                        print 'sample_raw_result: ',sample_raw_result
                        # print sample_raw_result, column_dic
                        if sample_raw_result:
                            check_time = sample_raw_result[0]["date"]
                            # print dp
                            upath = unified_path_granularity.format(unified_process_date=dp.split("/")[2],
                                                                    unified_device_code=platforms[dp.split("/")[4]][
                                                                        "device_code"],
                                                                    unified_market_code=platforms[dp.split("/")[4]][
                                                                        "market_code"],
                                                                    unified_process_hour=dp.split("/")[3],
                                                                    unified_granularity=granularity)
                            try:
        
                                unified_result = spark.read.parquet(upath).filter("review_id='{}'".format(sample_raw_result[0]["id"])).take(2)
                            except AnalysisException as e:
                                logger.info(
                                    'the path is not exist for unified level!!! expected unified path is {}'.format(
                                        upath))
        
                            for key, value in column_dic.items():
                                check_value(key, sample_raw_result[0][key], unified_result[0][value], p)
        
                    elif len(raw_result) == 0:
                        print 'No data'
                        continue


def check_value(index, exp, act, p):
    print index, exp, act, p
    try:
        if exp:
            if index == 'rating' or index == "product_id":
                exp = int(exp)
                act = int(act)
                if exp == act:
                    pass
                elif exp != act:
                    logger.info(
                        'number NOT Equal!!! !!!!!!!! index is {} , path is: {}  value is {} #~~~~# {}'.format(index, p,
                                                                                                               exp,
                                                                                                               act))
            elif index == 'user_purchased':
                exp = str(exp).lower()
                act = str(act).lower()
                if exp == act:
                    pass
                if exp != act:
                    logger.info(
                        'NOT Equal!!! !!!!!!!! index is {} , path is: {}  value is {} #~~~~# {}'.format(index, p, exp,
                                                                                                        act))
            else:
                if exp.lower().strip('"') == act.lower().strip('"'):
                    pass
                if exp != act:
                    logger.info(
                        'NOT Equal!!! !!!!!!!! index is {} , path is: {}  value is {} #~~~~# {}'.format(index, p, exp,
                                                                                                        act))

    except UnicodeEncodeError as e:
        logger.info(e)


# def main(spark, params):
#     # assert params.get('cluster_name'), _usage("cluster_name is required")
#     graularity = ["weekly", "daily", "hourly"]
#     for x in graularity:
#         logger.info("start to test granulairty as {}".format(x))
graularity = ["weekly", "daily", "hourly"]
for x in graularity:
    check_raw_unified_data_equals_with_granularity_for_one_day(x)
    logger.info("start to test granulairty as {}".format(x))



In [0]:

spark.read.csv("s3://prod_appannie_uniform_public_app_store_data/review-uniform/HOURLY/2019-02-14/00/gp/",sep="\t").filter("_c0='gp:AOqpTOF9guumkPxptpbPNMxsxuyxpSwgKy0QozyfIgqHczswqPDq4ThKXvFlf1-6Pom-jTJSsfH4sHAOoyrCAQU'").show(20,False)

spark.read.parquet("s3://b2c-prod-data-pipeline-unified-review/unified/review.v1/fact/process_granularity=hourly/process_date=2019-02-14/process_hour=0/device_code=android-all/market_code=google-play/").filter("review_id='gp:AOqpTOF9guumkPxptpbPNMxsxuyxpSwgKy0QozyfIgqHczswqPDq4ThKXvFlf1-6Pom-jTJSsfH4sHAOoyrCAQU'").show()

In [0]:

from pyspark.sql.functions import countDistinct
# spark.read.csv("s3://prod_appannie_uniform_public_app_store_data/review-uniform/MONTHLY/2017-01-31/gp/",sep="\t").filter("_c0='gp:AOqpTOHUKdAGUV_C94UVZHgqaXl9iqnCCdE-8Uqp3vdNW89Qt6UV9ADIOCo9P73sEEXBRFnDmALiiklCpHj9Gr4'").show(20,False,vertical=False)

print spark.read.parquet("s3://b2c-prod-data-pipeline-unified-review/unified/review.v1/fact/process_granularity=monthly/process_date=2017-03-31/").filter("(time<='2017-03-31' and time >='2017-03-01') and market_code='apple-store' and _identifier='220200118161416198'").cache().agg(countDistinct("review_id")).collect()


In [0]:

from datetime import datetime
from dateutil.relativedelta import relativedelta
from pyspark.storagelevel import StorageLevel
from pyspark.sql import functions as F
start_month = datetime.strptime("2017-06","%Y-%m")
end_month = datetime.strptime("2020-01","%Y-%m")
a_month = relativedelta(months=1)
now = start_month
while now <= end_month:
    path = "s3://b2c-prod-data-pipeline-unified-review/unified/review.v1/fact/process_granularity={weekly,daily,hourly}/process_date=%s-*/" % now.strftime("%Y-%m")
    print "%s started" % path 
    review_df = spark.read.option("basePath", "s3://b2c-prod-data-pipeline-unified-review/unified/review.v1/fact/").parquet(path).where("process_granularity in ('daily', 'weekly', 'hourly') and market_code in ('apple-store','google-play') and device_code in ('android-all', 'ios-all')")
    review_df = review_df.withColumn("event_month",F.from_unixtime(F.unix_timestamp("time", "yyyy-MM-dd"), "yyyy-MM")).withColumn("event_date",F.from_unixtime(F.unix_timestamp("time", "yyyy-MM-dd"), "yyyy-MM-dd")).repartition(1000, "event_date").drop("event_date")
    review_df.write.parquet("s3://b2c-prod-data-pipeline-unified-review/unified/review_load_temp/review_load_temp_v2/", mode="append", partitionBy=["event_month"])
    now += a_month
    print "%s ended" % path


In [0]:

for x in dup_list:
    result1 = spark.read.parquet('s3://b2c-prod-data-pipeline-unified-review/unified/review_load_temp/review_load_temp_v4/event_month=2019-09').filter("review_id='{}'".format(x["review_id"])).groupBy("process_date","process_hour","time").agg({"*":"count"}).orderBy(desc("time"),desc("process_date"),desc("process_hour")).take(1)
    result2 = spark.read.parquet("s3://b2c-prod-data-pipeline-unified-review/unified/review_load_temp/review_agg_v1/event_month=2019-09").filter("review_id='{}'".format(x["review_id"])).take(1)
    print result1,result2
    if result1[0]["process_date"]==result2[0]["process_date"] and result1[0]["process_hour"]==result2[0]["process_hour"] and result1[0]["time"]==result2[0]["time"]  :
        print 'pass'
    else:
        print 'failed!!!',result1, result2,x

for x in process_data_set:
    path = "s3://b2c-prod-data-pipeline-unified-review/unified/review.v1/fact/process_granularity={weekly,daily,hourly}/process_date=%s/" % x["process_date"].strftime("%Y-%m-%d")
    total_count = spark.read.option("basePath","s3://b2c-prod-data-pipeline-unified-review/unified/review.v1/fact/").parquet(path).where("market_code in ('apple-store','google-play')").count()
    if x["total_review"] != total_count:
       print "Not Equal!!!!!!!!", x["process_date"].strftime("%Y-%m-%d") ,x["total_review"], total_count
    else:
        print x["process_date"].strftime("%Y-%m-%d") ,x["total_review"],total_count